In [1]:
import math
from interval import interval, imath
from Polynomials import *

In [2]:
class Taylor:
    def __init__(self, poly:Poly, remainder:interval, domain:interval):
        self.P=poly
        self.I=interval(remainder)
        self.D=interval(domain)

    @property
    def order(self):
        return self.P.order
    
    def __repr__(self):
        return 'Taylor({0}, {1}, {2})'.format(self.P, self.I, self.D)
    
    def __add__(self, other):
        if isinstance(other,(int,float)):
            other=Taylor(Poly([other],0),0,self.D)
        D=self.D&other.D
        return Taylor(self.P+other.P,self.I+other.I,D)
    
    def __radd__(self,other):
        return self+other
    
    def __sub__(self, other):
        if isinstance(other,(int,float)):
            other=Taylor(Poly([other],0),0,self.D)
        D=self.D&other.D
        return Taylor(self.P-other.P,self.I-other.I,D)
    
    def __rsub__(self,other):
        if isinstance(other,(int,float)):
            other=Taylor(Poly([other],0),0,self.D)
        return other-self
    
    # full multiplication, n is the result order, defualt n is 'full'
    def fullmul(self,other,n='full'):
        if isinstance(other,(int,float)):
            if other==0:
                return 0
            else:
                return Taylor(other*self.P,other*self.I,self.D)
        D=self.D&other.D
        P=self.P*other.P
        I1=self.P.bound(D)*other.I
        I2=self.I*other.P.bound(D)
        I=I1+I2+self.I*other.I
        if n=='full':
            return Taylor(P,I,D)
        elif isinstance(n,int) and n>0 and n<=(self.n+other.n):
            return Taylor(P,I,D).truncation(n)
        else:
            raise Exception("n must be 'full' or integer between 0 ~ self.order+other.order")
    
    def __mul__(self, other):
        if isinstance(other,(int,float)):
            if other==0:
                return 0
            else:
                return Taylor(other*self.P,other*self.I,self.D)
        return self.fullmul(other).truncation(max(self.order,other.order))
    
    def __rmul__(self,other):
        return self*other
    
    def bound(self,c='best'):
        return self.P.bound(self.D,c)+self.I
    
    def __call__(self):
        return self.bound()
    
    def truncation(self,n,c='best'):
        P=self.P.truncation(n)
        I=(self.P-P).bound(self.D,c)+self.I
        return Taylor(P,I,self.D)
    
    def __truediv__(self, other):
        if isinstance(other,(int,float)):
            if other==0:
                raise Exception("Cannot dividied by zero!")
            else:
                return Taylor(self.P/other,self.I/other,self.D)
        else:
            raise Exception("Can only do scalar division!")
    
    def one(self):
        return Taylor(Poly.one(self.order),self.I,self.D)
    
    def zero(self):
        return Taylor(Poly.zero(self.order),self.I,self.D)
    
    def __eq__(self,other):
        if self.P==other.P and self.I==other.I and self.D==other.D:
            return True
    
    def square(self):
        return self*self
    
    def __pow__(self,n):
        if n==0:
            return self.one()
        elif n%2==0:
            if n==2:
                return self.square()
            else:
                p=int(n/2)
                return self.__pow__(p).square()
        elif n%2==1:
            if n==1:
                return self
            else:
                p=int((n-1)/2)
                return self*self.__pow__(p).square()
        else:
            raise Exception("n must be a positive integer")
    
    # 0~n+1 coefficients of exp sin cos
    def coeff(f,n,x,flag=0):
        if flag==1: # n+1 coefficients
            h=imath
        else: h=math
        if f=='exp':
            return h.exp(x)/math.factorial(n)
        if f=='sin':
            i=n%4
        if f=='cos':
            i=(n+1)%4
        if i==0:
            return h.sin(x)/math.factorial(n)
        elif i==1:
            return h.cos(x)/math.factorial(n)
        elif i==2:
            return -h.sin(x)/math.factorial(n)
        elif i==3:
            return -h.cos(x)/math.factorial(n)
    
    # elementary functions with lagrange remainder
    def element(f,n,x0,D):
        coef=[]
        for i in range(n+1):
            coef.append(Taylor.coeff(f,i,x0))
        coef1=Taylor.coeff(f,n+1,D,1)
        coef.append(coef1)
        I=coef1*((D-x0)**(n+1))
        return Taylor(Poly(coef,n),I,D)

    def compo_poly(self,p,c='best'):
        if c=='naive' or c=='n':
            return p.bound_naive(self)
        elif c=='Horner' or c=='H':
            return p(self)
        elif c=='best':
            if p.bound_naive(self).I&p(self).I==p(self).I:
                return p(self)
            else: return p.bound_naive(self)
        else:
            raise Exception("The options mush be naive(n), Horner(H) or best")
    #composition of a function 'f' and a Taylor model 'self'
    def compose1(self,f,n,x0):
        Bf=self.bound()
        Mg=Taylor.element(f,n,x0,Bf)
        M1=self.zero()
        M1.P.coef[1:]=self.P.coef[1:]
        M=M1.compo_poly(Mg.P,'best')
        return Taylor(M.P,M.I+Mg.I,self.D)
    
    # composition of two TM : f(g(x)),self=f,other=g        
    def compose(self,other,c='best'):
        if isinstance(other,Poly):
            other=Taylor(other,0,self.D)
        if c=='naive' or c=='n':
            return self.P.bound_naive(other)+self.zero()
        elif c=='Horner' or c=='H':
            return self.P(other)+self.zero()
        elif c=='best':
            if self.P.bound_naive(other).I&self.P(other).I==self.P(other).I:
                return self.P(other)+self.zero()
            else: return self.P.bound_naive(other)+self.zero()
        else:
            raise Exception("The options mush be naive(n), Horner(H) or best")

In [3]:
a=Taylor(Poly([0,1,2,3,4],4),interval([0,1]),interval([-1,1]))
b=Taylor(Poly([1,2,3,4,5],4),interval([0,2]),interval([-1,1]))
a

Taylor(Poly([0, 1, 2, 3, 4], 4), interval([0.0, 1.0]), interval([-1.0, 1.0]))

In [13]:
s=Taylor.element('sin',3,0,interval([-1.0, 1.0]))
s

Taylor(Poly([0.0, 1.0, -0.0, -0.16666666666666666, interval([-0.035061291033662366, 0.035061291033662366])], 3), interval([-0.035061291033662366, 0.035061291033662366]), interval([-1.0, 1.0]))

In [14]:
s2=s.compose(Taylor(Poly([0,2],3),interval([0,0]),interval([-1,1])))
s2

Taylor(Poly([0.0, 2.0, 0.0, -1.3333333333333333], 3), interval([-0.035061291033662366, 0.035061291033662366]), interval([-1.0, 1.0]))

In [15]:
Poly([0.0, 1.0, -0.0, -0.16666666666666666, interval([-0.035061291033662366, 0.035061291033662366])], 4)(interval[-2,2])

interval([-3.8943139898719314, 3.8943139898719314])

In [11]:
a.truncation(2)

Taylor(Poly([0, 1, 2], 2), interval([-3.0, 8.0]), interval([-1.0, 1.0]))

In [12]:
a.compose(Poly([0,2,0,0,0],4),'H')

Taylor(Poly([0, 2, 8, 24, 64], 4), interval([-0.0, 1.0]), interval([-1.0, 1.0]))

In [31]:
o=Taylor(Poly([0,2],1),0,interval([-1,1]))
a.P(o)

TaylorModel(Poly([0, 2], 1), interval([-88.0, 96.0]), interval([-1.0, 1.0]))

In [32]:
a.P(Poly([0,2],1))

Poly([0, 2, 8, 24, 64], 4)

In [50]:
interval([-0.035061291033662366, 0.035061291033662366])*16

interval([-0.5609806565385979, 0.5609806565385979])

In [52]:
b.compose(a)

TaylorModel(Poly([1, 2, 7, 22, 67], 4), interval([-37541.0, 78818.0]), interval([-1.0, 1.0]))

In [53]:
Taylor(Poly([0,2],3),interval([0,0]),interval([-1,1])).compose1('sin',3,0)

interval([-0.04166666666666667, 0.04166666666666667])


TaylorModel(Poly([0.0, 2.0, 0.0, -1.3333333333333333], 3), interval([-0.6666666666666667, 0.6666666666666667]), interval([-1.0, 1.0]))

In [54]:
interval([-0.035061291033662366, 0.035061291033662366])*16

interval([-0.5609806565385979, 0.5609806565385979])

In [55]:
c=Taylor.element('cos',3,0,interval([-1.0, 1.0]))
c

interval([0.022512596077839148, 0.04166666666666667])


TaylorModel(Poly([1.0, -0.0, -0.5, 0.0], 3), interval([0.0, 0.04166666666666667]), interval([-1.0, 1.0]))

In [59]:
a=Taylor(Poly([0,1,2,3,4],4),interval([0,1]),interval([-1,1]))

In [60]:
a.bound()

interval([-4.0, 11.0])

In [61]:
Mg=Taylor.element('sin',4,0,interval([-4.0, 11.0]))
Mg

interval([-0.008333333333333335, 0.008333333333333335])


TaylorModel(Poly([0.0, 1.0, -0.0, -0.16666666666666666, 0.0], 4), interval([-1342.0916666666672, 1342.0916666666672]), interval([-4.0, 11.0]))

In [62]:
a

TaylorModel(Poly([0, 1, 2, 3, 4], 4), interval([0.0, 1.0]), interval([-1.0, 1.0]))

In [63]:
M1=a

In [64]:
Mg.P

Poly([0.0, 1.0, -0.0, -0.16666666666666666, 0.0], 4)

In [65]:
M=M1.compo_poly(Mg.P,'best')
M

TaylorModel(Poly([0.0, 1.0, 2.0, 2.8333333333333335, 3.0], 4), interval([-220.66666666666666, 108.16666666666667]), interval([-1.0, 1.0]))

In [66]:
Taylor(M.P,M.I+Mg.I,interval([-1,1]))

TaylorModel(Poly([0.0, 1.0, 2.0, 2.8333333333333335, 3.0], 4), interval([-1562.758333333334, 1450.258333333334]), interval([-1.0, 1.0]))

In [6]:
b.P(a)

Taylor(Poly([1, 2, 7, 22, 67], 4), interval([-37541.0, 78816.0]), interval([-1.0, 1.0]))

In [13]:
b.compose(a)

Taylor(Poly([1, 2, 7, 22, 67], 4), interval([-37541.0, 78818.0]), interval([-1.0, 1.0]))